In [1]:
from utils import context_encoding, preprocessing

import torch
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv(r"./dataset/dataset.csv")
df.head()

,Scalar_B,BX_GSE,BY_GSM,BZ_GSM,SW_Temp,SW_Density,SW_Speed,Flow_Pressure,E_Field,Dst,time
0,7.8,-5.6,3.2,-4.0,76351.0,5.4,389.0,1.54,1.56,6,1640995200
1,8.3,-5.3,4.9,0.4,67379.0,4.7,390.0,1.26,-0.16,-3,1640998800
2,9.3,-2.7,3.9,-4.1,103254.0,2.1,454.0,0.87,1.86,-7,1641002400
3,9.8,1.7,4.0,-8.4,194757.0,2.7,466.0,1.17,3.91,-13,1641006000
4,8.5,-3.0,4.6,-4.4,126079.0,1.9,454.0,0.70,2.00,-21,1641009600


In [3]:
feature_cols = ['Scalar_B', 'BX_GSE', 'BY_GSM', 'BZ_GSM', 'SW_Temp', 'SW_Density', 'SW_Speed', 'Flow_Pressure', 'E_Field']
target_col = 'Dst'
context_cols = ['time']
numerical_features_info = {k:"standard" for k in feature_cols}

dataset = preprocessing.TabularDataset(
    data_path=r"./dataset/dataset.csv",
    feature_cols=feature_cols,
    target_col=target_col,
    context_cols=context_cols,
    numerical_features_info=numerical_features_info
)

In [4]:
train_dataset, val_dataset, test_dataset = preprocessing.split_data(dataset)
train_loader = preprocessing.create_dataloader(train_dataset, batch_size=8)
val_loader = preprocessing.create_dataloader(val_dataset)
test_loader = preprocessing.create_dataloader(test_dataset)

In [5]:
sample = next(iter(train_loader))
print(sample)

{'features': tensor([[-0.2493, -0.2198, -0.2300, -0.2356, -0.1504, -0.1540, -0.1286, -0.2366,
         -0.2455],
        [-0.2228, -0.2162, -0.2659, -0.2233, -0.1973, -0.1149, -0.1424, -0.1639,
         -0.2504],
        [ 4.3124,  4.3123,  4.3120,  4.3123, -0.1600, -0.1556, -0.1684, -0.2534,
          4.0478],
        [-0.2360, -0.2316, -0.2128, -0.2461, -0.1334, -0.1540, -0.0751, -0.2182,
         -0.2404],
        [-0.2379, -0.2130, -0.2250, -0.2311, -0.0888, -0.1524, -0.0800, -0.2143,
         -0.2474],
        [-0.2360, -0.2444, -0.2314, -0.2511, -0.1615, -0.1438, -0.1408, -0.2193,
         -0.2396],
        [-0.2466, -0.2325, -0.2328, -0.2174, -0.1903, -0.0757, -0.2122, -0.1622,
         -0.2515],
        [-0.2296, -0.2371, -0.2041, -0.2324, -0.1564, -0.1407, -0.1838, -0.2299,
         -0.2469]]), 'target': tensor([-10.,  14.,  -2., -28.,  -9.,  -7.,   6., -18.]), 'context': {'time': tensor([1.7092e+09, 1.7081e+09, 1.6901e+09, 1.6627e+09, 1.6514e+09, 1.6628e+09,
        1.7087e+0

In [14]:
sample = next(iter(train_loader))

context_cols = {"time": "time"}

encoder_model = context_encoding.ContextEncoder(
    context_cols,
    embedding_dim=32
)

print(sample["context"].items())

input_context = torch.concat([v for k, v in sample['context'].items()], dim=0)

encoder = encoder_model(input_context)

encoder.shape

dict_items([('time', tensor([1.6586e+09, 1.6909e+09, 1.6556e+09, 1.7240e+09, 1.7080e+09, 1.7098e+09,
        1.6997e+09, 1.6519e+09]))])


torch.Size([8, 32])